# Imports and Setup

In [4]:
!git clone https://github.com/cis700/hw1-release.git
!mv hw1-release/dills/* .
!mv hw1-release hw1
!pip install pymagnitude
!pip install nltk
!pip install symspellpy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch
import time
import nltk
from nltk.metrics.distance import edit_distance, jaccard_distance
import threading
import random
from nltk.stem import PorterStemmer
from pymagnitude import *
import datetime
import pickle
from tqdm import tqdm
from symspellpy.symspellpy import SymSpell, Verbosity
from pymagnitude import *
import ast



from hw1.helper import Logger


##MISSING NLTK DOWNLOAD
nltk.download("brown")

fatal: destination path 'hw1-release' already exists and is not an empty directory.
mv: cannot stat 'hw1-release/dills/*': No such file or directory
mv: cannot move 'hw1-release' to 'hw1/hw1-release': Directory not empty
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

# Tensorboard

In [0]:
#! rm -r ./logs
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

# Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')
local_dir= 'drive/My Drive/CIS700-004/project'
%cd 'drive/My Drive/CIS700-004/project'

# Data Generator

In [0]:
#initialize spell suggester
sym_spell = SymSpell(3, 6, compact_level=0)
sym_spell.load_dictionary("frequency_dictionary_en_82_765.txt", 0, 1)

#max number of candidates to consider as a replacement for a word
candLimit = 100

#list of all parts of speech which are allowed to be replaced when part-of-speech filtering is on
switch_tags = ["JJ", "JJR", "JJS", "NN", "NNS", "RB", "RBR", "RBS", "VBD", "VBG", "VBN", "VBP", "VBZ"]

#list of all nltk pos tags (other than punctuation)
all_pos = ["CC", "DT", "EX", "IN", "JJ", "JJR", "JJS", "MD", "NN", "NNS", "NNP", "NNPS", "PDT", "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB"]

def corrupt(tokens, tags, dist_threshold=3, pos_filter=True):
    """replaces (num_words) words in the (sentence) with 
    similar words that are only (dist_threshold) away, 
    with optional filtering by part-of-speech"""
    
    #initialize labels (1 for correct words, 0 for out-of-place word)
    labels = [1]*len(tags)
    
    #gets indices of words that can be replaced (must have suitable part-of-speech), 
    #or all indices of real words (non-punctuation/symbols) if pos_filter is false
    replaceables = getReplaceables(tags, pos_filter)
    real_word = ""
    
    #if there are words that can be replaced, replace one
    if(len(replaceables)>0):
        replace_ind = random.choice(replaceables)
        real_word = tokens[replace_ind]
        fake_word = suggestWord(real_word, tags[replace_ind], dist_threshold, pos_filter=pos_filter)
        if(fake_word!=real_word):
                labels[replace_ind] = 0
                tokens[replace_ind] = fake_word
    return tokens, labels, real_word

def getReplaceables(pos_tags, filtering):
    """returns indices of tokens that can be replaced (restricted set if pos_filtering is on)"""
    if(filtering):
      replaceables = [i for i,t in enumerate(pos_tags) if t in switch_tags]
    else:
      replaceables = [i for i,t in enumerate(pos_tags) if t in all_pos]
    return replaceables

def suggestWord(real_word, real_tag, e_thresh=3, pos_filter=True):
    """suggests a word at most edit distance e_thresh away, with optional pos_filtering
    (enforces that new word has same pos as original word)"""
    
    #get similarly spelled words
    suggestions = sym_spell.lookup(real_word, Verbosity.ALL, e_thresh)
    
    #initialize list of word candidates and frequencies
    candidates = [""] * len(suggestions)
    candFreq = [0]*len(suggestions)
    numSuggest = 0
    
    #loop through all word candidates
    for s in suggestions:
        
        #add the current word to the candidate list if it has correct pos, or if pos filtering is off
        tag=""
        if(pos_filter):
            _, tag = nltk.pos_tag([s.term])[0]
        if((tag==real_tag or (not pos_filter)) and s.term!=real_word):  
            candidates[numSuggest] = s.term
            candFreq[numSuggest] = s.count
            numSuggest+=1
            
            #stop searching if maximum number of candidates is reached
            if(numSuggest>candLimit):
                break
                
    #if there is a suitable replacement word, return it; otherwise, return original word
    if(numSuggest>0):
        return random.choices(candidates[0:numSuggest], weights=candFreq[0:numSuggest], k=1)[0]
    else:
        return real_word


def createData(output_file, num_cycles, pos_filter):
    """samples sentences from brown corpus for num_cycles, and writes corrupted data to output_file."""
    
    #loads brown corpus
    brown = nltk.corpus.brown
    allFiles = brown.fileids()
    
    #max number of sentences to pull from any given file at a time
    sent_per_file = 20
    
    #min sentence length to be considered valid sentence
    sent_threshold = 5
    
    with open(output_file, 'a') as f:
        for j in range(num_cycles):
            
            #randomly choose file and get pos-tagged sentences 
            currentFile = random.choice(allFiles)            
            sentences = brown.tagged_sents(currentFile)
            sentences = [s for s in sentences if len(s)>=sent_threshold]
            
            for i in tqdm(range(sent_per_file)):
                #randomly choose sentence, corrupt it, 
                #then write sentence, label, and real word to output file
                tagged_sent = random.choice(sentences)
                tokens, tags = zip(*tagged_sent)
                tokens, tags = list(tokens), list(tags)
                new_sent, labels, real_word = corrupt(tokens, tags, pos_filter=pos_filter)
                f.write(str(new_sent)+"\n"+str(labels)+"\n"+real_word+"\n")

In [10]:
sym_spell = SymSpell(3, 6, compact_level=0)
sym_spell.load_dictionary("freqdict.pkl", 0, 1)

suggestions = sym_spell.lookup("dog", Verbosity.ALL, 3)
print(suggestions)
with open("freqdict.pkl", "r") as f:
  i=0
  for line in f:
    print(f)

[]


In [0]:
##EDIT TO CORRECT FILE CONVENTIONS
createData("train_filtered.txt", 2000, True)
createData("test_filtered.txt", 2000, True)
createData("train_unfiltered.txt", 2000, True)
createData("test_unfiltered.txt", 2000, True)

In [0]:
def getDataset(filename, savename):
    """Converts corrupted sentences to vector representations (using pymagnitude), saving to output file"""
    
    #load magnitude embeddings
    vectors =  Magnitude("GoogleNews-vectors-negative300.magnitude")
    
    sentences = []
    labels = []
    with open(filename, "r") as f:
      
        #iterates through file, converts each word in sentence to vector, 
        #builds sentences and labels lists
        for line in tqdm(f):
            sentence_words = ast.literal_eval(line)
            label = ast.literal_eval(f.readline())
            real_word = f.readline()[0:-1]
            sentence_vector = []
            for word in sentence_words:
                sentence_vector.append(vectors.query(word))
            sentences.append(sentence_vector)
            labels.append(label)
                
    #saves sentence vector and label objects as pickle files
    ##EDIT TO CORRECT FILE CONVENTION
    pickle.dump(sentences, open(savename+"sentvectors.pkl", "wb"))
    pickle.dump(labels, open(savename+"labelvectors.pkl", "wb"))

In [0]:
##RUN ON EACH DATASET

# LSTM

In [0]:
class LSTM(nn.Module): 
    def __init__(self, input_size, hidden_size, num_layers, bidirectional= False):
        super(LSTM, self).__init__()
        self.hidden_size= hidden_size
        self.num_layers= num_layers
        self.num_directions = 2 if bidirectional else 1
        self.lstm= nn.LSTM(input_size, hidden_size, 
                           num_layers, batch_first= True, bidirectional= bidirectional)
        self.fc= nn.Linear(hidden_size*self.num_directions, 1)
        
        #for logging
        self.step = 0
        self.epoch = 0
    
    def forward(self, x): 
        h0 = torch.zeros(self.num_directions*self.num_layers, 1, self.hidden_size).cuda()
        lstm_out, _ = self.lstm(x, (h0, c0))
        out = lstm_out.view(-1, lstm_out.size(2))
        out = F.sigmoid(self.fc(F.relu(out)))
        return out

    def train(self, train_data, train_labels, max_epochs, lr= .00001):
        
        #initialize objects
        logger = Logger("./"+modelname+"_logs")
        optimizer = torch.optim.Adam(self.parameters(), lr= lr)
        
        #BCE loss since predicting binary tag for each word
        criterion = torch.nn.BCELoss()
        
        for epoch in range(max_epochs):
            
            #randomize order of examples
            order = torch.randperm(len(train_data))
            for i in tqdm(range(len(train_data))):
                
                #read next sentence and labels from dataset
                sentence = train_data[order[i]]
                word_labels = train_labels[order[i]]

                #rearrange sentence and labels for feeding into network
                inputs = torch.tensor(sentence).view(1, len(sentence), -1).type(torch.FloatTensor).cuda()
                labels = torch.tensor([word_labels]).transpose(0,1).float().cuda()

                #backprop
                optimizer.zero_grad()
                y_pred = self.forward(inputs)
                loss = criterion(y_pred, labels)
                loss.backward()
                optimizer.step()

                #log loss
                logger.scalar_summary("loss", loss.item(), self.step)
                logger.writer.flush()
                self.step +=1

            torch.save(self, modelname +"/epoch" + str(self.epoch))
            self.epoch +=1

In [0]:
##TRAIN LSTMS (4)

# Evaluation

In [0]:
def evaluate(sentence_file, text_file, output_file, model):
    results = []
    sentence_vects = pickle.load(open(sentence_file, "rb"))
    with open(text_file, "r") as f:
        i=0
        with torch.no_grad():
          
            #for each example, record the sentence, vectorized sentence, true labels, and predictions
            for line in tqdm(f):
              
                #read in example
                sentence_arr = ast.literal_eval(line)
                label = ast.literal_eval(f.readline())
                real_word = f.readline()[0:-1]
                sentence_vect = sentence_vects[i]
                
                #get model predictions
                tensor_inputs = torch.tensor(sentence_vect).view(1, len(sentence_vect), -1).type(torch.FloatTensor).cuda()
                out = model.forward(tensor_inputs)
                out = out.view(-1)
                out=out.cpu().tolist()
                
                #discretize predictions, with threshold at .5
                pred = [1 if j>.5 else 0 for j in out]
                
                #get which words were predicted to be out-of-place, and which ones actually were
                predicted_inapprop = [sentence_arr[j] for j in range(len(sentence_arr)) if pred[j]<.5]
                actual_inapprop = [sentence_arr[j] for j in range(len(sentence_arr)) if label[j]==0]
                
                #append to results list
                results.append({"sentence":sentence_arr, 
                                "label":label, 
                                "output":out, 
                                "discrete": pred,
                                "inapprop":actual_inapprop, 
                                "pred_inapprop":predicted_inapprop})
                i+=1
    pickle.dump(results, open(output_file, "wb"))
    return

In [0]:
##RUN ON ALL MODELS, 2 DATASETS